# Zadanie rekrutacyjne 4 - Wycena instrumentów

## Opis zadania

Twoim zadaniem jest obliczenie wartości bieżącej netto (NPV) i podstawowej miary ryzyka stóp procentowych (BPV) dla transakcji walutowego swapu stopy procentowej (CIRS) według stanu na dzień 17 lutego 2023.

Dane wejściowe do zadania zostały przekazane w formie plików CSV i obejmują:
- `transakcje.csv` - stylizowane parametry transakcji do wyceny,
- `krzywa_forward_EUR.csv` - stylizowana krzywa forward dla waluty EUR wyliczona na 17 lutego 2023,
- `krzywa_forward_PLN.csv` - stylizowana krzywa forward dla waluty PLN wyliczona na 17 lutego 2023,
- `krzywa_discount_EUR.csv` - stylizowana krzywa dyskontowa dla waluty EUR wyliczona na 17 lutego 2023,
- `krzywa_discount_PLN.csv` - stylizowana krzywa dyskontowa dla waluty PLN wyliczona na 17 lutego 2023,

Twoim zadaniem jest wykonanie obliczeń zgodnie z algorytmem opisanym poniżej. Jako wynik otrzymasz wartości NPV i BPV, osobne dla każdej nogi każdej transakcji. Zapoznaj się z opisem wszystkich obliczeń, a następnie przygotuj kod pozwalający rozwiązać zadanie.

Wykonując obliczenia przyjmij następujące konwencje długości roku:
- dla waluty PLN: ACT/ACT - do obliczeń przyjmuje się rzeczywisty czas pomiędzy wydarzeniami i rzeczywistą długość roku,
- dla waluty EUR: 30/360 - na cele obliczeń rok składa się z 12 miesięcy, z których każdy liczy 30 dni.

## Zależności

Zaimportuj pakiety, które będą Ci potrzebne w czasie rozwiązywania zadania.

In [195]:
import pandas as pd
import numpy as np
from datetime import datetime
import ast

## Import danych

Zaimportuj dane wejściowe z plików dostarczonych wraz z zadaniem. Opis zawartości każdego z pliku znajdziesz powyżej.

Krzywe wyceny mają formę:
|krzywa|tenor|data konstrukcji krzywej|data zapadalności|stopa zerokuponowa|czynnik dyskontowy|
|---|---|---|---|---|---|

Transakcje są dostarczone w formie tabeli:
|nr transakcji|data wyceny|strona transkcji|waluta|nominał|krzywa forwardowa|krzywa dyskontowa|daty płatności|okres odsetkowy|oprocentowanie w pierwszym okresie|data zapadalności|
|---|---|---|---|---|---|---|---|---|---|---|

Każda z transakcji obejmuje dwie nogi, każda w innej walucie i wyceniana w oparciu o zmienną stopę oprocentowania, odpowiadające stronie sprzedającej (S) i kupującej (B) transakcji. Zgodnie z konwencją, __dla strony kupującej przepływy pieniężne mają wartość ujemną, a dla strony sprzedającej - wartość dodatnią__.


Dla każdej transakcji, w kolumnie "daty płatności" znajduje się lista dni, w których kończą się poszczególne okresy odsetkowe transakcji. __Przyjmij, że data zakończenia okresu jest jednocześnie datą rozpoczęcia kolejnego__. Wraz z końcem ostatniego okresu odsetkowego, następuje również wypłata nominału transakcji. W wycenie instrumentu uwzględniane są jedynie przyszłe przepływy pieniężne, dlatego istotne są jedynie daty wypłat następujące po dacie wyceny.


Na podstawie tabeli transakcji __przygotuj harmonogram płatności__ zawierający daty rozpoczęcia i zakończenia poszczególnych okresów odsetkowych, który wykorzystasz w toku dalszych obliczeń.

In [196]:
transakcje_df  = pd.read_csv('transactions.csv', sep=';')
krzywa_forward_EUR_df = pd.read_csv('krzywa_forward_EUR.csv',sep=';')
krzywa_forward_PLN_df = pd.read_csv('krzywa_forward_PLN.csv',sep=';')
krzywa_discount_EUR_df = pd.read_csv('krzywa_discount_EUR.csv',sep=';')
krzywa_discount_PLN_df = pd.read_csv('krzywa_discount_PLN.csv',sep=';')

In [197]:
transakcje_df.dtypes

nr transakcji                          object
data wyceny                            object
strona transakcji                      object
waluta                                 object
nominał                                 int64
krzywa forwardowa                      object
krzywa dyskontowa                      object
daty płatności                         object
okres odsetkowy                        object
oprocentowanie w pierwszym okresie     object
data zapadalności                      object
Unnamed: 11                           float64
dtype: object

In [198]:
transakcje_df.head()

,nr transakcji,data wyceny,strona transakcji,waluta,nominał,krzywa forwardowa,krzywa dyskontowa,daty płatności,okres odsetkowy,oprocentowanie w pierwszym okresie,data zapadalności,Unnamed: 11
0,CIRS_A,'2023-02-17',S,EUR,1000000,forward_EUR,discount_EUR,"['2023-01-17', '2023-04-17', '2023-07-17', '20...",3M,1.817%,'2025-01-15',NaN
1,CIRS_A,'2023-02-17',B,PLN,4520000,forward_PLN,discount_PLN,"['2023-01-17', '2023-04-17', '2023-07-17', '20...",3M,7.370%,'2025-01-15',NaN
2,CIRS_B,'2023-02-17',B,EUR,1010098,forward_EUR,discount_EUR,"['2022-11-23', '2023-02-23', '2023-05-23', '20...",3M,2.288%,'2027-08-23',NaN
3,CIRS_B,'2023-02-17',S,PLN,4751000,forward_PLN,discount_PLN,"['2022-11-23', '2023-02-23', '2023-05-23', '20...",3M,6.950%,'2027-08-23',NaN


In [199]:
import re
# Przekształcenie kolumny 'okres odsetkowy' na wartość liczbową
def transform_period(period):
    match = re.match(r'(\d+)([YM])', period)
    if match:
        number = int(match.group(1))
        unit = match.group(2)
        if unit == 'M':
            return number  # konwersja na lata dla miesięcy
        elif unit == 'Y':
            return number*12
    return period
transakcje_df['okres odsetkowy'] = transakcje_df['okres odsetkowy'].apply(transform_period)

transakcje_df['oprocentowanie w pierwszym okresie'] = transakcje_df['oprocentowanie w pierwszym okresie'].str.rstrip('%').astype(float) / 100.0

transakcje_df['data zapadalności'] = pd.to_datetime(transakcje_df['data zapadalności'], errors='coerce')
transakcje_df['data wyceny'] = pd.to_datetime(transakcje_df['data wyceny'], errors='coerce')

if 'Unnamed: 11' in transakcje_df.columns:
    transakcje_df.drop(columns=['Unnamed: 11'], inplace=True)

In [200]:
transakcje_df.head()

,nr transakcji,data wyceny,strona transakcji,waluta,nominał,krzywa forwardowa,krzywa dyskontowa,daty płatności,okres odsetkowy,oprocentowanie w pierwszym okresie,data zapadalności
0,CIRS_A,2023-02-17,S,EUR,1000000,forward_EUR,discount_EUR,"['2023-01-17', '2023-04-17', '2023-07-17', '20...",3,0.01817,2025-01-15
1,CIRS_A,2023-02-17,B,PLN,4520000,forward_PLN,discount_PLN,"['2023-01-17', '2023-04-17', '2023-07-17', '20...",3,0.07370,2025-01-15
2,CIRS_B,2023-02-17,B,EUR,1010098,forward_EUR,discount_EUR,"['2022-11-23', '2023-02-23', '2023-05-23', '20...",3,0.02288,2027-08-23
3,CIRS_B,2023-02-17,S,PLN,4751000,forward_PLN,discount_PLN,"['2022-11-23', '2023-02-23', '2023-05-23', '20...",3,0.06950,2027-08-23


In [201]:
transakcje_df['daty płatności'] = transakcje_df['daty płatności'].apply(ast.literal_eval)

def generate_payment_schedule(payment_dates):
    schedule = []
    for i in range(1, len(payment_dates)):
        schedule.append({
            'start_date': payment_dates[i-1],
            'end_date': payment_dates[i]
        })
    return schedule

transakcje_df['payment_schedule'] = transakcje_df['daty płatności'].apply(generate_payment_schedule)

In [202]:
transakcje_df[['nr transakcji', 'payment_schedule']].head()

,nr transakcji,payment_schedule
0,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202..."
1,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202..."
2,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202..."
3,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202..."


## Obliczenie czynników dyskontowych

Czynnik dyskontowy $DF_{t_{0}, t_{1}}$ dla okresu pomiędzy datami $t_{0}$ a $t_{1}$ jest funkcją wykładniczą stopy zerokuponowej na ten sam okres ($ZC_{t_{0}, t_{1}}$) przeskalowanej przez frakcję roku YF, dla której jest liczony.
$$
DF_{t_{0}, t_{1}} = e^{-ZC_{t_{0}, t_{1}}\cdot YF} \tag{1}
$$ (label:1)

Frakcja roku to długość okresu $\Delta_{i}$ pomiędzy datami $t_{0}$ a $t_{1}$ ($\Delta_{t_{0}, t_{1}} = t_{1} - t_{0}$) podzielona przez przyjętą długość roku Y:
$$
YF_{t_{0}, t_{1}} = \frac{\Delta_{t_{0}, t_{1}}}{Y} \tag{2}
$$

Użyte w obliczeniach __wartości $\Delta_{i}$ oraz $Y$ będą zależały od konwencji dlugości roku obowiązującej dla danej waluty, określonej we wstępie do ćwiczenia__.

Krzywa dyskontowa zawiera wartości stopy zerokuponowej i obliczone czynniki dyskontowe dla wybranych dat (tenorów). Jeżeli tenor pokrywa się z datą wypłaty dla wybranej transakcji, wartości współczynników dyskontowych można odczytać bezpośrednio z krzywej. W pozostałych przypadkach, dla daty t wypadającej pomiędzy datami zapadalności $T_{0}$ i $T_{1}$ uwzględnionymi na krzywej wyceny, należy dokonać __interpolacji liniowej__ wartości stopy zerokuponowej:
$$
ZC_{T} = ZC_{T_{0}} + (ZC_{T_{1}} - ZC_{T_{0}}) \cdot \frac{t - T_{0}}{T_{1} - T_{0}} \tag{3}
$$

i podstawić otrzymaną wartość do równania (1), aby obliczyć czynnik dyskontowy.




Dla podanych transakcji, należy obliczyć wartości czynników dyskontowych dla dat podanych w kolumnie `daty płatności` w poniższy sposób:
1. Na odpowiedniej dla danej waluty __krzywej dyskontowej__ znajdź tenory, pomiędzy którymi wypada wybrana data płatności (lub z którymi się pokrywa).
2. Wyznacz stopę zerokuponową dla daty wypłaty przy użyciu równania (3).
3. Podstaw otrzymaną wartość do równania (1) podstawiając jako datę końca okresu wybraną datę płatności, a jako datę początku okresu - datę wyceny.

__Pamiętaj, że wartości stopy zeroprocentowej w tabeli są podane w %__.

In [203]:
def calculate_discount_factor(zero_rate, start_date, end_date):
    delta = (end_date - start_date).days / 365.0
    discount_factor = np.exp(-zero_rate / 100 * delta)
    return discount_factor

def interpolate_zero_rate(date, t0, t1, zc_t0, zc_t1):
    delta_t0_t1 = (t1 - t0).days / 365.0
    delta_t0_t = (date - t0).days / 365.0
    delta_t1_t = (t1 - date).days / 365.0
    
    zero_rate = zc_t0 + (zc_t1 - zc_t0) * (delta_t0_t / delta_t0_t1)
    return zero_rate

def find_tenors(discount_curve, payment_date):
    tenors = discount_curve['data zapadalności']
    tenors = pd.to_datetime(tenors, format='%d/%m/%Y')
    suitable_tenors = []
    for i in range(len(tenors)):
        if tenors[i] >= payment_date:
            suitable_tenors.append(i)
    return suitable_tenors

def calculate_discount_factors(transaction, discount_curve):
    payment_schedule = transaction['payment_schedule']
    currency = transaction['waluta']
    
    discount_factors = []
    for period in payment_schedule:
        start_date = period['start_date']
        end_date = period['end_date']
        end_date = pd.to_datetime(end_date,format='%Y-%m-%d')
        
        
        suitable_tenors = find_tenors(discount_curve, end_date)
        
        if len(suitable_tenors) == 1:
            # Bez interpolacji, jeśli znaleziono dokładne dopasowanie
            zero_rate = discount_curve.loc[suitable_tenors[0], 'stopa zerokuponowa']
        elif len(suitable_tenors) > 1:
            # Interpolacja, jeśli znaleziono więcej niż jedno dopasowanie
            t0 = discount_curve.loc[suitable_tenors[0], 'data zapadalności']
            t0 = pd.to_datetime(t0,format='%d/%m/%Y')
            t1 = discount_curve.loc[suitable_tenors[1], 'data zapadalności']
            t1 = pd.to_datetime(t1,format='%d/%m/%Y')
            zc_t0 = discount_curve.loc[suitable_tenors[0], 'stopa zerokuponowa']
            zc_t1 = discount_curve.loc[suitable_tenors[1], 'stopa zerokuponowa']
            zero_rate = interpolate_zero_rate(end_date, t0, t1, zc_t0, zc_t1)
        else:
            raise ValueError(f"Nie znaleziono odpowiednich tenorów dla daty {end_date}.")
        
        discount_factor = calculate_discount_factor(zero_rate, transaction['data wyceny'], end_date)
        discount_factors.append(discount_factor)
    
    return discount_factors

transakcje_df['discount_factors'] = transakcje_df.apply(lambda row: calculate_discount_factors(row, krzywa_discount_EUR_df if row['waluta'] == 'EUR' else krzywa_discount_PLN_df), axis=1)
transakcje_df[['nr transakcji', 'payment_schedule', 'discount_factors']]


,nr transakcji,payment_schedule,discount_factors
0,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...","[0.9957805648061879, 0.9878300507752062, 0.979..."
1,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...","[0.9879714225927552, 0.970060693976514, 0.9522..."
2,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...","[0.9996055707066062, 0.9928346318981531, 0.984..."
3,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...","[0.9988021008496258, 0.9809179438039141, 0.962..."


## Obliczenie stóp procentowych

Stopa procentowa jest określana wraz z rozpoczęciem każdego okresu odsetkowego. 
Dla biegnącego okresu, stopa procentowa jest podana dla każdej nogi transakcji w kolumnie `oprocentowanie w pierwszym okresie`.
Dla okresów przyszłych, należy ją wyznaczyć na podstawie krzywych forward na dzień rozpoczęcia okresu

Krzywa forward podaje wartości stóp zerokuponowych i czynników dyskontowych na konkretne daty. Pomiędzy tymi datami, wartości należy wyznaczyć przez interpolację. 
Do wyceny instrumentów, wykorzystaj __interpolację liniową__, zgodnie z równaniami (1) i (3). Aby obliczyć stopy oprocentowania dla przyszłych okresów odsetkowych, wykonaj ponoiższe obliczenia:

1. Dla pierwszego (biegnącego) okresu odsetkowego, stopa oprocentowania jest znana i podana w kolumnie `oprocentowanie w pierwszym okresie` w danych wejściowych.
Dla pozostałych okresów:
2. Zgodnie z metodologią przestawioną w poprzednim punkcie, na podstawie odpowiedniej dla danej waluty __krzywej forwardowej__ wyznacz czynniki dyskontowe $DF_{start}$ dla okresu pomiędzy __datą wyceny__ a __datą rozpoczęcia__ każdego okresu odsetkowego. __Nie możesz tego wykonać dla biegnącego okresu odsetkowego, którego data rozpoczęcia wypada przed datą wyceny (bądącą równocześnie datą konstrukcji krzywej wyceny)__.
3. Analogicznie, na podstawie __krzywej forwardowej__ wyznacz czynniki dyskontowe $DF_{end}$ dla okresu pomiędzy __datą wyceny__ a __datą zakończenia__ każdego okresu odsetkowego.
4. Dla każdego okresu odsetkowego zaczynającego się w $t_{start}$ i kończącego w $t_{end}$, oblicz interpolowaną stopę oprocentowania $r_{t_{start}, t_{end}}$ na podstawie wyznaczonych współczynników dyskontowych:
$$
r_{t_{start}, t_{end}} = (\frac{DF_{start}}{DF_{end}} - 1) \cdot YF_{t_{start}, t_{end}} \cdot 100\% \tag{4}
$$

__Pamiętaj, że wartości stopy zeroprocentowej w tabeli są podane w %__.





In [204]:
def find_tenors(discount_curve, payment_date):
    tenors = discount_curve['data zapadalności']
    tenors = pd.to_datetime(tenors, format='%d/%m/%Y')
    suitable_tenors = []
    for i in range(len(tenors)):
        if tenors[i] >= payment_date:
            suitable_tenors.append(i)
    return suitable_tenors

def calculate_interest_rate(transaction, discount_curve, forward_curve):
    payment_schedule = transaction['payment_schedule']
    currency = transaction['waluta']
    data_wyceny = transaction['data wyceny']
    data_wyceny = pd.to_datetime(data_wyceny, format='%Y-%m-%d')
    
    interest_rates = []
    for period in payment_schedule:
        start_date = period['start_date']
        start_date = pd.to_datetime(start_date, format='%Y-%m-%d')
        end_date = period['end_date']
        end_date = pd.to_datetime(end_date, format='%Y-%m-%d')
        
        # Dla bieżącego okresu odsetkowego
        if start_date == data_wyceny:
            interest_rate = transaction['oprocentowanie w pierwszym okresie']
        
        else:
            # Wyznacz czynniki dyskontowe DF_start i DF_end
            suitable_tenors_start = find_tenors(forward_curve, start_date)
            suitable_tenors_end = find_tenors(forward_curve, end_date)
            
            # Znajdź odpowiednie tenory na krzywej forwardowej
            t_start = forward_curve.loc[suitable_tenors_start[0], 'data zapadalności']
            t_start = pd.to_datetime(t_start, format='%d/%m/%Y')
            t_end = forward_curve.loc[suitable_tenors_end[0], 'data zapadalności']
            t_end = pd.to_datetime(t_end, format='%d/%m/%Y')
            
            DF_start = calculate_discount_factor(forward_curve.loc[suitable_tenors_start[0], 'stopa zerokuponowa'], (data_wyceny), (t_start))
            DF_end = calculate_discount_factor(forward_curve.loc[suitable_tenors_end[0], 'stopa zerokuponowa'], (data_wyceny), end_date)
            
            # Oblicz frakcję roku YF dla okresu od t_start do t_end
            delta_t_start_t_end = ((t_end) - (t_start)).days / 365.0
            YF_t_start_t_end = delta_t_start_t_end
            
            # Oblicz stopę procentową dla okresu od t_start do t_end
            interest_rate = ((DF_start / DF_end) - 1) * YF_t_start_t_end * 100
        
        interest_rates.append(interest_rate)
    
    return interest_rates

transakcje_df['interest_rates'] = transakcje_df.apply(lambda row: calculate_interest_rate(row, krzywa_discount_EUR_df if row['waluta'] == 'EUR' else krzywa_discount_PLN_df, krzywa_forward_EUR_df if row['waluta'] == 'EUR' else krzywa_forward_PLN_df), axis=1)

transakcje_df[['nr transakcji', 'payment_schedule', 'interest_rates']]


,nr transakcji,payment_schedule,interest_rates
0,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...","[0.06725562416148294, 0.19391809856632208, 0.2..."
1,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...","[0.1742020319498995, 0.5526244516965274, 0.268..."
2,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...","[0.00048036552970056505, 0.2105615248141133, 0..."
3,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...","[0.0012478545118562098, 0.7597755896805816, 0...."


## Obliczenie przepływów pieniężnych

Dla każdego okresu odsetkowego zaczynającego się w $t_{0}$ i kończącego w $t_{1}$, przepływ pieniężny CF jest równy nominałowi nogi transakcji przemnożonemu przez stopę procentową dla danego okresu i frakcję roku odpowiadającą jego długości. 

Kierunek przepływu (znak otrzymanej wartości) zależy od strony transakcji. Jest to uwzględnione przez współczynnik $s_{B/S}$.
$$
CF_{t{0}, t_{1}} = s_{B/S} \cdot N \cdot r_{t_{0}, t_{1}} \cdot YF_{t_{0}, t_{1}} \tag{5}
$$.

Wykonaj obliczenia przepływów pieniężnych przyjmując, że nominał nie ulega amortyzacji - jego wartość jest taka sama przez cały okres trwania umowy.

In [205]:
def calculate_cash_flows(transaction, discount_curve, forward_curve):
    payment_schedule = transaction['payment_schedule']
    data_wyceny = pd.to_datetime(transaction['data wyceny'], format='%Y-%m-%d')
    nominal_value = transaction['nominał']
    side_factor = 1 if transaction['strona transakcji'] == 'B' else -1
    
    cash_flows = []
    
    for period in payment_schedule:
        start_date = pd.to_datetime(period['start_date'], format='%Y-%m-%d')
        end_date = pd.to_datetime(period['end_date'], format='%Y-%m-%d')
        
        interest_rates = calculate_interest_rate(transaction, discount_curve, forward_curve)
        
        delta_t_start_end = (end_date - start_date).days / 365.0
        interest_rate = interest_rates[payment_schedule.index(period)]
        
        cash_flow = side_factor * nominal_value * interest_rate * delta_t_start_end
        
        cash_flows.append(cash_flow)
    
    return cash_flows

transakcje_df['cash_flows'] = transakcje_df.apply(lambda row: calculate_cash_flows(row, krzywa_discount_EUR_df if row['waluta'] == 'EUR' else krzywa_discount_PLN_df, krzywa_forward_EUR_df if row['waluta'] == 'EUR' else krzywa_forward_PLN_df), axis=1)

transakcje_df[['nr transakcji', 'payment_schedule', 'cash_flows']]

,nr transakcji,payment_schedule,cash_flows
0,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...","[-16583.578560365655, -48346.70402612414, -553..."
1,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...","[194151.74410197017, 622754.7656762073, 302701..."
2,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...","[122.30108491888298, 51860.85474838376, 41469...."
3,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...","[-1494.321162455492, -880171.9193560203, -1252..."


## Obliczenie NPV

Wartość bieżąda netto (NPV) transakcji CIRS na dzień wyceny ($t_{pricing}$) to suma przepływów pieniężnych pomnożonych przez czynnik dyskontowy obliczony na dzień ich wypłaty ($DF_{T_{CF}}$). Dodatkowo, w raz z wypłatą ostatniego przepływu odsetkowego w dniu zakończenia transakcji ($t_{last}$), następuje wypłata nominału $N$, którego zdyskontowana (na tę datę) wartość jest uwzględniona w wycenie. Ostatecznie, wzór na NPV przyjmuje postać:

$$
NPV = s_{B/S} \cdot N \cdot DF_{t_{pricing}, t_{last}} + \sum_{t_{CF, 1}}^{t_{last}}  CF_{t_{CF}} \cdot DF_{t_{pricing}, t_{CF}} \tag{6}
$$

Wykonaj odpowiednie obliczenia dla każdej z nóg obydwu transakcji.

In [206]:
def calculate_npv(transaction, discount_curve, forward_curve):
    payment_schedule = transaction['payment_schedule']
    currency = transaction['waluta']
    data_wyceny = pd.to_datetime(transaction['data wyceny'], format='%Y-%m-%d')
    nominal_value = transaction['nominał']
    side_factor = 1 if transaction['strona transakcji'] == 'B' else -1
    
    npv = 0.0

    last_payment_date = pd.to_datetime(payment_schedule[-1]['end_date'], format='%Y-%m-%d')
    
    DF_t_pricing_last = calculate_discount_factor(discount_curve.loc[0, 'stopa zerokuponowa'], data_wyceny, last_payment_date)
    
    for period in payment_schedule:
        start_date = pd.to_datetime(period['start_date'], format='%Y-%m-%d')
        end_date = pd.to_datetime(period['end_date'], format='%Y-%m-%d')
        
        interest_rates = calculate_interest_rate(transaction, discount_curve, forward_curve)
        
        delta_t_start_end = (end_date - start_date).days / 365.0
        interest_rate = interest_rates[payment_schedule.index(period)]
        
        cash_flow = side_factor * nominal_value * interest_rate * delta_t_start_end
        
        DF_t_pricing_CF = calculate_discount_factor(discount_curve.loc[0, 'stopa zerokuponowa'], data_wyceny, end_date)
        
        npv += cash_flow * DF_t_pricing_CF
    
    npv += side_factor * nominal_value * DF_t_pricing_last
    
    return npv

npvs = []
for idx, transaction in transakcje_df.iterrows():
    npv = calculate_npv(transaction, krzywa_discount_EUR_df if transaction['waluta'] == 'EUR' else krzywa_discount_PLN_df, krzywa_forward_EUR_df if transaction['waluta'] == 'EUR' else krzywa_forward_PLN_df)
    npvs.append(npv)
transakcje_df['NPV'] = npvs
transakcje_df[['nr transakcji', 'payment_schedule', 'NPV']]


,nr transakcji,payment_schedule,NPV
0,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...",-1.278778e+06
1,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...",6.056402e+06
2,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...",1.214715e+06
3,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...",-4.813582e+06


## Obliczenie BPV

Aby obliczyć podstawową miarę ryzyka stóp procentowych dla transakcji (BPV), przygotuj przesunięte krzywe wyceny __podnosząc wartości stopy zerokuponowej na każdej krzywej dyskontowej i forwardowej o 1 punkt bazowy ($1 bp = 0.01\%$)__. Na podstawie tak wyznaczonej stopy zerokuponowej, przelicz dla krzywej __współczyynnik dyskontowy zgodnie z równaniem (1)__. Pozostawiająć __pozostałe wartości niezmienione, powtórz wszystkie kroki oblieczeń opisane powyżej z użyciem przesuniętych krzywych__.  Otrzymasz w ten sposób wartość $NPV_{+1bp}$.

Miarę ryzyka BPV obliczysz, odejmując od otrzymanej wartości wcześniej obliczone NPV transakcji.

$$
BPV = NPV_{+1bp}-NPV \tag{7}
$$

Wykonaj odpowiednie obliczenia.

In [207]:
def calculate_shifted_discount_factor(r_shifted, date_pricing, date):
    delta_t = (date - date_pricing).days / 365.0
    DF_shifted = 1 / ((1 + r_shifted / 100) ** delta_t)
    return DF_shifted

def calculate_npv_shifted(transaction, discount_curve, forward_curve, r_shifted):
    payment_schedule = transaction['payment_schedule']
    currency = transaction['waluta']
    data_wyceny = pd.to_datetime(transaction['data wyceny'], format='%Y-%m-%d')
    nominal_value = transaction['nominał']
    side_factor = 1 if transaction['strona transakcji'] == 'B' else -1
    
    npv_shifted = 0.0
    
    last_payment_date = pd.to_datetime(payment_schedule[-1]['end_date'], format='%Y-%m-%d')
    
    DF_t_pricing_last_shifted = calculate_shifted_discount_factor(discount_curve.loc[0, 'stopa zerokuponowa'] + r_shifted, data_wyceny, last_payment_date)

    for period in payment_schedule:
        start_date = pd.to_datetime(period['start_date'], format='%Y-%m-%d')
        end_date = pd.to_datetime(period['end_date'], format='%Y-%m-%d')
        
        interest_rates = calculate_interest_rate(transaction, discount_curve, forward_curve)
        
        delta_t_start_end = (end_date - start_date).days / 365.0
        interest_rate = interest_rates[payment_schedule.index(period)]

        cash_flow = side_factor * nominal_value * interest_rate * delta_t_start_end

        DF_t_pricing_CF_shifted = calculate_shifted_discount_factor(discount_curve.loc[0, 'stopa zerokuponowa'] + r_shifted, data_wyceny, end_date)

        npv_shifted += cash_flow * DF_t_pricing_CF_shifted

    npv_shifted += side_factor * nominal_value * DF_t_pricing_last_shifted
    
    return npv_shifted

bpvs = []
for idx, transaction in transakcje_df.iterrows():
    npv = calculate_npv(transaction, krzywa_discount_EUR_df if transaction['waluta'] == 'EUR' else krzywa_discount_PLN_df, krzywa_forward_EUR_df if transaction['waluta'] == 'EUR' else krzywa_forward_PLN_df)
    
    npv_shifted = calculate_npv_shifted(transaction, krzywa_discount_EUR_df if transaction['waluta'] == 'EUR' else krzywa_discount_PLN_df, krzywa_forward_EUR_df if transaction['waluta'] == 'EUR' else krzywa_forward_PLN_df, 0.01)

    bpv = npv_shifted - npv
    bpvs.append(bpv)

transakcje_df['BPV'] = bpvs

transakcje_df[['nr transakcji', 'payment_schedule', 'BPV']]


,nr transakcji,payment_schedule,BPV
0,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...",-399.779711
1,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...",25735.498426
2,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...",862.142971
3,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...",-49717.057299


## Wyeksportuj wyniki

Jako wynik zadania, wyeksportuj do plików CSV:
- wyniki obliczeń w formie tabeli zawierającą zestawione wartości NPV i BPV dla każdej nogi każdej z transakcji,
- harmonogramy dla każdej nogi każdej transakcji, zawierające:
    - datę rozpoczęcia okresu odsetkowego,
    - datę zakończenia okresu odsetkowego,
    - stopę oprocentowania dla danego okresu,
    - czynnik dyskontowy na koniec okresu,
    - przepływ pieniężny dla danego okresu.

W pakiecie podsumowującym zadanie, prześlij:
- otrzymane dane wejściowe,
- plik Jupyter Notebook zawierający kod rozwiązania zadania oraz wyniki obliczeń,
- wyeksportowane pliki wynikowe.

__Pamiętaj, aby skonstruować tabele i nazwać pliki tak, aby było jasne, której transakcji dotyczy.__

In [208]:
save_df=transakcje_df[[
    "nr transakcji",
    "payment_schedule",
    "interest_rates",
    "discount_factors",
    "cash_flows",
    "BPV",
    "NPV",]
]
save_df.to_csv("wyniki.csv")
save_df

,nr transakcji,payment_schedule,interest_rates,discount_factors,cash_flows,BPV,NPV
0,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...","[0.06725562416148294, 0.19391809856632208, 0.2...","[0.9957805648061879, 0.9878300507752062, 0.979...","[-16583.578560365655, -48346.70402612414, -553...",-399.779711,-1.278778e+06
1,CIRS_A,"[{'start_date': '2023-01-17', 'end_date': '202...","[0.1742020319498995, 0.5526244516965274, 0.268...","[0.9879714225927552, 0.970060693976514, 0.9522...","[194151.74410197017, 622754.7656762073, 302701...",25735.498426,6.056402e+06
2,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...","[0.00048036552970056505, 0.2105615248141133, 0...","[0.9996055707066062, 0.9928346318981531, 0.984...","[122.30108491888298, 51860.85474838376, 41469....",862.142971,1.214715e+06
3,CIRS_B,"[{'start_date': '2022-11-23', 'end_date': '202...","[0.0012478545118562098, 0.7597755896805816, 0....","[0.9988021008496258, 0.9809179438039141, 0.962...","[-1494.321162455492, -880171.9193560203, -1252...",-49717.057299,-4.813582e+06
